# Wan2GP Native-Mode Validation (Kaggle/Colab)

This notebook is a **validation-focused** variant to test the zero-optimization/native behavior:
- BF16 non-quantized default for Wan2.2 I2V A14B
- Auto-quantization disabled
- Compile disabled
- Attention forced to SDPA
- Profile forced to 1
- Step-skipping cache disabled

Run cells in order.

## 1) Confirm GPU accelerator is enabled

- Colab: `Runtime -> Change runtime type -> GPU`
- Kaggle: `Settings -> Accelerator -> GPU`

In [ ]:
import os
import subprocess

try:
    subprocess.run(["nvidia-smi"], check=True)
except Exception as exc:
    hint = (
        "Kaggle: open Settings -> Accelerator and select GPU."
        if "KAGGLE_KERNEL_RUN_TYPE" in os.environ
        else "Colab: open Runtime -> Change runtime type and select GPU."
    )
    raise RuntimeError(f"GPU not detected. Enable GPU acceleration, then rerun this cell. {hint}") from exc

## 2) Configure paths

In [ ]:
import os
from pathlib import Path

IS_KAGGLE = "KAGGLE_KERNEL_RUN_TYPE" in os.environ or Path("/kaggle").exists()
WAN2GP_ROOT = Path("/kaggle/working/Wan2GP" if IS_KAGGLE else "/content/Wan2GP").resolve()

print(f"Runtime: {'Kaggle' if IS_KAGGLE else 'Colab'}")
print(f"WAN2GP_ROOT: {WAN2GP_ROOT}")

## 3) Clone or update repository

In [ ]:
import subprocess

# Change this if your native-mode changes live in another repo/branch.
REPO_URL = "https://github.com/kanaya23/Wan2GP.git"

if WAN2GP_ROOT.exists():
    print("Repository exists, pulling latest changes...")
    subprocess.run(["git", "-C", str(WAN2GP_ROOT), "pull"], check=True)
else:
    subprocess.run(["git", "clone", REPO_URL, str(WAN2GP_ROOT)], check=True)

## 4) Install dependencies

In [ ]:
import os
import subprocess
import sys

env = os.environ.copy()
env.setdefault("DEBIAN_FRONTEND", "noninteractive")

apt_prefix = ["apt-get"] if IS_KAGGLE else ["sudo", "apt-get"]
subprocess.run(apt_prefix + ["update", "-qq"], check=True, env=env)
subprocess.run(
    apt_prefix
    + [
        "install",
        "-y",
        "--no-install-recommends",
        "ffmpeg",
        "libglib2.0-0",
        "libgl1",
        "libportaudio2",
    ],
    check=True,
    env=env,
)

subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"], check=True, env=env)
subprocess.run(
    [
        sys.executable,
        "-m",
        "pip",
        "install",
        "torch==2.8.0",
        "torchvision",
        "torchaudio",
        "--index-url",
        "https://download.pytorch.org/whl/cu128",
    ],
    check=True,
    env=env,
)
subprocess.run(
    [
        sys.executable,
        "-m",
        "pip",
        "install",
        "xformers==0.0.32.post2",
        "--index-url",
        "https://download.pytorch.org/whl/cu128",
    ],
    check=True,
    env=env,
)
subprocess.run([sys.executable, "-m", "pip", "install", "-r", str(WAN2GP_ROOT / "requirements.txt")], check=True, env=env)
subprocess.run([sys.executable, "-m", "pip", "install", "hydra-core", "omegaconf"], check=True, env=env)

print("Dependencies installed.")

## 5) Static native-mode checks (fast)

In [ ]:
        import json

        def fail(msg):
            raise AssertionError(msg)

        finetune_file = WAN2GP_ROOT / "finetunes" / "i2v_2_2.json"
        if not finetune_file.exists():
            fail(f"Missing override file: {finetune_file}")

        data = json.loads(finetune_file.read_text())
        model = data.get("model", {})

        if model.get("auto_quantize", None) is not False:
            fail("finetunes/i2v_2_2.json must set model.auto_quantize to false")

        for key in ("URLs", "URLs2", "text_encoder_URLs"):
            urls = model.get(key, [])
            if not urls:
                fail(f"Missing model.{key} in finetunes/i2v_2_2.json")
            bad = [u for u in urls if "quanto" in u.lower() and "int8" in u.lower()]
            if bad:
                fail(f"model.{key} contains quantized URLs: {bad}")

        wgp_text = (WAN2GP_ROOT / "wgp.py").read_text()
        required_snippets = [
            'attention_mode = "sdpa"',
            'compile = ""',
            'skip_steps_cache_type = ""',
            'quantizeTransformer = False',
            'def compute_profile(override_profile, output_type="video"):
    return 1',
        ]

        missing = [s for s in required_snippets if s not in wgp_text]
        if missing:
            fail("Missing expected native-mode code snippets:
" + "
".join(missing))

        print("PASS: static native-mode checks.")

## 6) Runtime smoke check (fast)

In [ ]:
import os
import subprocess
import sys
import time

cmd = [
    sys.executable,
    "-u",
    "wgp.py",
    "--i2v-14B",
    "--attention",
    "sage2",
    "--compile",
    "--test",
]

expected = {
    "attention": "Ignoring attention mode 'sage2': native mode enforces sdpa",
    "compile": "PyTorch compilation is disabled in native mode",
}
found = {k: False for k in expected}

print("Running:", " ".join(cmd))
p = subprocess.Popen(
    cmd,
    cwd=str(WAN2GP_ROOT),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    env=os.environ.copy(),
)

deadline = time.time() + 180

while time.time() < deadline and p.poll() is None and not all(found.values()):
    line = p.stdout.readline()
    if not line:
        time.sleep(0.2)
        continue
    print(line, end="")
    for key, token in expected.items():
        if token in line:
            found[key] = True

if p.poll() is None:
    p.terminate()
    try:
        p.wait(timeout=10)
    except Exception:
        pass

missing = [k for k, v in found.items() if not v]
if missing:
    print("\nFAILED runtime smoke check. Missing:", missing)
    print("Tip: verify you cloned the branch that contains native-mode changes.")
    raise AssertionError("Runtime smoke check failed")

print("\nPASS: runtime smoke check.")

## 7) Optional deep check (slow, large download)

In [ ]:
import os
import subprocess
import sys

RUN_DEEP_CHECK = False  # set True only if you want full model resolution check

if RUN_DEEP_CHECK:
    cmd = [sys.executable, "-u", "wgp.py", "--i2v-14B", "--test"]
    out = subprocess.run(
        cmd,
        cwd=str(WAN2GP_ROOT),
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        env=os.environ.copy(),
    )
    print(out.stdout)
    if "Native mode resolved files for i2v_2_2:" not in out.stdout:
        raise AssertionError("Did not find resolved-files native log line")
    if "quanto" in out.stdout.lower() and "int8" in out.stdout.lower():
        raise AssertionError("Detected quanto/int8 in deep-check output")
    print("PASS: deep check")
else:
    print("Deep check skipped (set RUN_DEEP_CHECK=True to run).")

## 8) Optional UI launch

In [ ]:
import sys
print("Launch command:")
print(f"cd {WAN2GP_ROOT} && {sys.executable} wgp.py --listen --server-port 7860")
print("Native mode is already enforced by code-level defaults/guards.")